In [ ]:
# Importar las librerías necesarias para el análisis
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import json
import warnings

# Ignorar advertencias (warnings) para mantener una salida limpia
warnings.filterwarnings("ignore")

# Cargar el dataframe limpio.
df_cleaned = pd.read_csv('datos/df_cleaned.csv')

# Haremos una copia para mantener el original intacto.
df_featured = df_cleaned.copy()

### Sección 5: Ingeniería de Características (con Discretización Robusta)

#### Resumen Ejecutivo de la Sección

Esta sección transforma el `df_cleaned` en un formato listo para el modelado, utilizando un **enfoque funcional y robusto** para la discretización de variables continuas.

1.  **Discretización mediante Función Ordinal:** En lugar de un mapa codificado, se ha implementado una función `get_ordinal_rank`. Esta función asigna un rango numérico (0 para la mejor calidad, números más altos para la peor) buscando palabras clave (ej. 'Excelente', 'Apta', 'Dura') dentro de las etiquetas de calidad. Este método es robusto y evita los errores de `NaN` causados por variaciones en el texto de las etiquetas.
2.  **One-Hot Encoding de `CONTAMINANTES`:** La columna de texto `CONTAMINANTES` se transformó en columnas binarias para cada tipo de contaminante, permitiendo al modelo interpretar cada uno como una característica independiente.
3.  **Eliminación de Columnas Redundantes y No Informativas:** Se eliminaron las columnas de alta cardinalidad, identificadores, la columna `CONTAMINANTES` original y las columnas de medición continua originales, ya que ahora están representadas por sus contrapartes ordinales (discretizadas).

El resultado es un DataFrame `df_featured` completamente numérico y verificado, sin valores `NaN`, listo para el modelado.

In [ ]:
# --- 1. Discretización: Función Robusta para Codificación Ordinal ---

def get_ordinal_rank(label):
    """Asigna un rango ordinal a una etiqueta de calidad buscando palabras clave."""
    if not isinstance(label, str):
        return None # Manejar valores no textuales
    
    label = label.lower() # Convertir a minúsculas para consistencia
    
    # Rango 3 (Peor Calidad)
    if 'muy dura' in label or 'muy alta' in label:
        return 3
    # Rango 2
    if 'no apta' in label or 'mala calidad' in label or 'dura' in label or 'salina' in label or 'alta' in label:
        return 2
    # Rango 1
    if 'apta como faap' in label or 'aceptable' in label or 'ligeramente' in label or 'media' in label or 'presencia' in label:
        return 1
    # Rango 0 (Mejor Calidad)
    if 'excelente' in label or 'buena calidad' in label or 'suave' in label or 'dulce' in label or 'baja' in label or 'ausencia' in label:
        return 0
    
    return -1 # Devolver un valor por defecto para identificar etiquetas no mapeadas

# Identificar y aplicar la función a las columnas de calidad
quality_cols = [col for col in df_featured.columns if col.startswith('CALIDAD_')]
for col in quality_cols:
    df_featured[col] = df_featured[col].apply(get_ordinal_rank)
print(f"Se ha aplicado la función de codificación ordinal a {len(quality_cols)} columnas.")

# Verificación de NaN y valores no mapeados: Asegurar que la codificación fue exitosa
assert df_featured[quality_cols].isnull().sum().sum() == 0, "Se generaron valores NaN durante la codificación."
unmapped_check = (df_featured[quality_cols] == -1).sum().sum()
assert unmapped_check == 0, f"Se encontraron {unmapped_check} etiquetas de calidad no mapeadas (valor -1)."
print("Verificación exitosa: No se generaron valores NaN ni se encontraron etiquetas no mapeadas.")

# --- 2. One-Hot Encoding de la columna 'CONTAMINANTES' ---
contaminant_list = df_featured['CONTAMINANTES'].str.split(', ').explode().unique()
unique_contaminants = sorted([c for c in contaminant_list if c != 'Sin Contaminantes'])
for contaminant in unique_contaminants:
    new_col_name = f"CONTAMINANTE_{contaminant}"
    df_featured[new_col_name] = df_featured['CONTAMINANTES'].apply(lambda x: 1 if contaminant in x else 0)
print(f"Se crearon {len(unique_contaminants)} nuevas columnas para contaminantes.")

# --- 3. Eliminación de Columnas No Relevantes o Redundantes ---
cols_to_drop = [
    'CLAVE', 'SITIO', 'ORGANISMO_DE_CUENCA', 'ESTADO', 'MUNICIPIO', 'ACUIFERO', 'SUBTIPO', 'PERIODO',
    'CONTAMINANTES',
    'ALC_mg/L', 'CONDUCT_mS/cm', 'SDT_M_mg/L', 'FLUORUROS_mg/L', 'DUR_mg/L', 
    'COLI_FEC_NMP/100_mL', 'N_NO3_mg/L', 'AS_TOT_mg/L', 'CD_TOT_mg/L', 'CR_TOT_mg/L', 
    'HG_TOT_mg/L', 'PB_TOT_mg/L', 'MN_TOT_mg/L', 'FE_TOT_mg/L'
]
df_featured.drop(columns=cols_to_drop, inplace=True, errors='ignore')
print("Se eliminaron columnas no relevantes o redundantes.")

# --- 4. Codificación Binaria de Columnas de Cumplimiento y Semáforo ---
compliance_cols = [col for col in df_featured.columns if 'CUMPLE_CON' in col or 'SEMAFORO' in col]
binary_mapping = {'SI': 0, 'NO': 1, 'VERDE': 0, 'AMARILLO': 1, 'ROJO': 1}
for col in compliance_cols:
    df_featured[col] = df_featured[col].map(binary_mapping)
print("Se codificaron binariamente las columnas de cumplimiento y semáforo.")

# --- Verificación Final ---
print(f"\nForma final del DataFrame: {df_featured.shape}")
print("Vista previa del DataFrame transformado:")
display(df_featured.head())